# Описание проекта

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Описание данных
  Данные геологоразведки трёх регионов находятся в файлах:
- geo_data_0.csv.
- geo_data_1.csv.
- geo_data_2.csv.
- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

### Условия задачи:
- Для обучения модели подходит только линейная регрессия (остальные — недостаточно предсказуемые).
- При разведке региона исследуют 500 точек, из которых выбирают 200 лучших для расчёта прибыли.
- Бюджет на разработку скважин в регионе — 10 млрд рублей.
- Один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.
- После оценки рисков нужно оставить лишь те регионы, в которых вероятность убытков меньше 2.5%. Среди них выбирают регион с наибольшей средней прибылью.

Данные синтетические: детали контрактов и характеристики месторождений не разглашаются.

# 1. Загрузка и подготовка данных

- Загрузите и подготовьте данные. Поясните порядок действий.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse

from sklearn.model_selection import GridSearchCV
from sklearn.utils import shuffle
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score

data_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_2 = pd.read_csv('/datasets/geo_data_2.csv')
data_0.info()
data_1.info()
data_2.info()
print(data_0.head())
print(data_1.head())
print(data_2.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8

- пропусков в столбцах не обнаружено

### Проверим на дубли

In [ ]:
def duble(data):
    result = (data[data.duplicated(subset='id', keep=False) == True]
              .sort_values(by='id',ascending=False))
    return result

In [ ]:
duble(data_0)

,id,f0,f1,f2,product
90815,fiKDv,0.049883,0.841313,6.394613,137.346586
16633,fiKDv,0.157341,1.028359,5.585586,95.817889
41724,bxg6G,-0.823752,0.546319,3.630479,93.007798
1364,bxg6G,0.411645,0.856830,-3.653440,73.604260
97785,bsk9y,0.378429,0.005837,0.160827,160.637302
89582,bsk9y,0.398908,-0.400253,10.122376,163.433078
92341,TtcGQ,0.110711,1.022689,0.911381,101.318008
60140,TtcGQ,0.569276,-0.104876,6.440215,85.350186
21426,Tdehs,0.829407,0.298807,-0.049563,96.035308
75715,Tdehs,0.112079,0.430296,3.218993,60.964018


In [ ]:
duble(data_1)

,id,f0,f1,f2,product
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305
2721,bfPNe,-9.494442,-5.463692,4.006042,110.992147
82178,bfPNe,-6.202799,-4.820045,2.995107,84.038886
1305,LHZR0,11.170835,-1.945066,3.002872,80.859783
41906,LHZR0,-8.989672,-4.286607,2.009139,57.085625
5849,5ltQ6,-3.435401,-12.296043,1.999796,57.085625
84461,5ltQ6,18.213839,2.191999,3.993869,107.813044


In [ ]:
duble(data_2)

,id,f0,f1,f2,product
28039,xCHr8,1.633027,0.368135,-2.378367,6.120525
43233,xCHr8,-0.847066,2.101796,5.597130,184.388641
44378,Vcm5J,-1.229484,-2.439204,1.222909,137.968290
95090,Vcm5J,2.587702,1.986875,2.482245,92.327572
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420
45404,KUPhW,0.231846,-1.698941,4.990775,11.716299
55967,KUPhW,1.211150,3.176408,5.543540,132.831802


In [ ]:
data = pd.merge(pd.merge(data_0, data_1, how='outer'), data_2, how='outer')
data.shape

(300000, 5)

In [ ]:
# Проверим повторяется ли уникальный идентификатор скважины (id) в разных регионах
duble_all = data[data['id'].isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index)]
duble_all.pivot_table(index=['id'])

,f0,f1,f2,product
id,,,,
2tyMi,-0.606461,-0.885092,-4.283302,107.597060
5ltQ6,7.389219,-5.052022,2.996832,82.449334
5ssQt,-0.828471,0.525571,1.141407,86.657395
74z30,0.913209,0.073436,6.071940,134.207410
A5aEY,-0.110142,0.546210,-0.942456,61.134785
AGS9W,0.260476,-0.181729,-1.486258,72.800479
CXJ2r,3.197496,-0.907634,3.154519,116.298430
D8TNs,-0.182942,1.864224,2.947016,72.538120
G6k8A,1.344636,1.482481,1.191256,70.639495


 - Количество дубликатов и анамальных значений(повторяются в разных регионах) небольшое, поэтому удалим их

In [ ]:
data_0_n = data_0[(~data_0['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]
data_1_n = data_1[(~data_1['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]
data_2_n = data_2[(~data_2['id']
                     .isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index))]

data = data[~data['id'].isin(data['id'].value_counts()[data['id'].value_counts() >= 2].index)]

In [ ]:
data_0_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99953 entries, 0 to 99999
Data columns (total 5 columns):
id         99953 non-null object
f0         99953 non-null float64
f1         99953 non-null float64
f2         99953 non-null float64
product    99953 non-null float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [ ]:
data_1_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99978 entries, 0 to 99999
Data columns (total 5 columns):
id         99978 non-null object
f0         99978 non-null float64
f1         99978 non-null float64
f2         99978 non-null float64
product    99978 non-null float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


In [ ]:
data_2_n.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99971 entries, 0 to 99999
Data columns (total 5 columns):
id         99971 non-null object
f0         99971 non-null float64
f1         99971 non-null float64
f2         99971 non-null float64
product    99971 non-null float64
dtypes: float64(4), object(1)
memory usage: 4.6+ MB


## Вывод

  - в результате предобработки данных пропусков не обнаружено
  - найдены и удалены дубликаты уникальных идентификаторов скважин в каждом регионе
  - найдены и удалены повторяющиеся уникальные идентификаторы скважин в разных регионах (это аномалии)

# 2. Обучение и проверка модели

- 2.1. Разобъем данные на обучающую и валидационную выборки в соотношении 75:25.

In [ ]:
sample_pred = []
sample_target = []

state = np.random.RandomState(12345)

features_0 = data_0_n.drop(['product', 'id'], axis=1)
target_0 = data_0_n['product']
features_0_train, features_0_valid, target_0_train, target_0_valid  = train_test_split(features_0,
                                                                                       target_0,
                                                                                       test_size=.25, 
                                                                                       random_state=state)
features_1 = data_1_n.drop(['product', 'id'], axis=1)
target_1 = data_1_n['product']
features_1_train, features_1_valid, target_1_train, target_1_valid  = train_test_split(features_1,
                                                                                       target_1,
                                                                                       test_size=.25, 
                                                                                       random_state=state)
features_2 = data_2_n.drop(['product', 'id'], axis=1)
target_2 = data_2_n['product']
features_2_train, features_2_valid, target_2_train, target_2_valid  = train_test_split(features_2,
                                                                                       target_2,
                                                                                       test_size=.25, 
                                                                                       random_state=state)

In [ ]:
print('Размер обучающей выборки r0', features_0_train.shape, '// Размер целевой выборки', target_0_train.shape)
print('Размер валидационной выборки r0', features_0_valid.shape, '// Размер целевой выборки', target_0_valid.shape)

Размер обучающей выборки r0 (74964, 3) // Размер целевой выборки (74964,)
Размер валидационной выборки r0 (24989, 3) // Размер целевой выборки (24989,)


In [ ]:
print('Размер обучающей выборки r1', features_1_train.shape, '// Размер целевой выборки', target_1_train.shape)
print('Размер валидационной выборки r1', features_1_valid.shape, '// Размер целевой выборки', target_1_valid.shape)

Размер обучающей выборки r1 (74983, 3) // Размер целевой выборки (74983,)
Размер валидационной выборки r1 (24995, 3) // Размер целевой выборки (24995,)


In [ ]:
print('Размер обучающей выборки r2', features_2_train.shape, '// Размер целевой выборки', target_2_train.shape)
print('Размер валидационной выборки r2', features_2_valid.shape, '// Размер целевой выборки', target_2_valid.shape)

Размер обучающей выборки r2 (74978, 3) // Размер целевой выборки (74978,)
Размер валидационной выборки r2 (24993, 3) // Размер целевой выборки (24993,)


<div class="alert alert-block alert-success">
<b>Успех:</b> Разбиение проведено корректно. Молодец, что напечатал размеры полученных наборов. Иногда это помогает избежать ошибок.
</div>

In [ ]:
def scaling_features(features_train, features_valid):
    scaler = StandardScaler()
    features_train = scaler.fit_transform(features_train)
    features_valid = scaler.transform(features_valid)
    return features_train, features_valid

features_0_train, features_0_valid = scaling_features(features_0_train, features_0_valid)
features_1_train, features_1_valid = scaling_features(features_1_train, features_1_valid)
features_2_train, features_2_valid = scaling_features(features_2_train, features_2_valid)

- 2.2. Обучим модель и сделаем предсказания на валидационной выборке. Подсчитаем MSE и RMSE модели

In [ ]:
def Lin_Reg(features_train, features_valid, target_train, target_valid, ind, region):
    model = LinearRegression()
    model.fit(features_train, target_train)
    prediction = model.predict(features_valid)
    result.loc[ind] = ['model_linear_'+ region,\
                       'Lin_Reg',\
                       mean_absolute_error(target_valid, prediction),\
                       mse(target_valid, prediction)**(.5), region]
    ind+=1
    return model, result, ind, prediction

In [ ]:
result = pd.DataFrame(columns=['name', 'type', 'mae', 'rmse', 'region'])
ind = 0

- предсказания для первого региона (geo_data_0)

In [ ]:
model_0, result, ind, pred_0 = Lin_Reg(features_0_train, features_0_valid,\
                                              target_0_train, target_0_valid, ind, 'r0')
result

,name,type,mae,rmse,region
0,model_linear_r0,Lin_Reg,31.249234,37.912922,r0


 - предсказания для второго региона (geo_data_1)

In [ ]:
model_1, result, ind, pred_1 = Lin_Reg(features_1_train, features_1_valid,\
                                              target_1_train, target_1_valid, ind, 'r1')
result

,name,type,mae,rmse,region
0,model_linear_r0,Lin_Reg,31.249234,37.912922,r0
1,model_linear_r1,Lin_Reg,0.718493,0.892664,r1


 - предсказания для третьего региона (geo_data_2)

In [ ]:
model_2, result, ind, pred_2 = Lin_Reg(features_2_train, features_2_valid,\
                                              target_2_train, target_2_valid, ind, 'r2')
result

,name,type,mae,rmse,region
0,model_linear_r0,Lin_Reg,31.249234,37.912922,r0
1,model_linear_r1,Lin_Reg,0.718493,0.892664,r1
2,model_linear_r2,Lin_Reg,32.887091,40.093195,r2


 - 2.3. Сделаем подсчёт среднего запаса сырья в каждом регионе

In [ ]:
def stock(row):
    if row.region == 'r0':
        return target_0.mean()*1000
    elif row.region == 'r1':
        return target_1.mean()*1000
    else:
        return target_2.mean()*1000
        
    
result['stock_of_raw_materials'] = result.apply(stock, axis=1)
result

,name,type,mae,rmse,region,stock_of_raw_materials
0,model_linear_r0,Lin_Reg,31.249234,37.912922,r0,92497.243831
1,model_linear_r1,Lin_Reg,0.718493,0.892664,r1,68824.460112
2,model_linear_r2,Lin_Reg,32.887091,40.093195,r2,94997.510094


2.4. Сохрани предсказания и правильные ответы на валидационной выборке.

In [ ]:
pred_0_s = pd.Series(pred_0, index=range(24989))
pred_1_s = pd.Series(pred_1, index=range(24995))
pred_2_s = pd.Series(pred_2, index=range(24993))

target_0_valid_s = pd.Series(target_0_valid).reset_index(drop=True)
target_1_valid_s = pd.Series(target_1_valid).reset_index(drop=True)
target_2_valid_s = pd.Series(target_2_valid).reset_index(drop=True)

In [ ]:
sample_pred = {'predictions_r0':pred_0_s,\
               'predictions_r1':pred_1_s,\
               'predictions_r2':pred_2_s}

sample_target = {'target_0_valid':target_0_valid_s,\
                 'target_1_valid':target_1_valid_s,\
                 'target_2_valid':target_2_valid_s}

zips = zip(sample_target.values(), sample_pred.values())
zips

In [ ]:
print(pred_0_s.index == target_0_valid_s.index)
print(pred_1_s.index == target_1_valid_s.index)
print(pred_2_s.index == target_2_valid_s.index)

[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]
[ True  True  True ...  True  True  True]


## Вывод

- Самый большой запас сырья в регионе r2, затем идет r0. Регион r1 - самый невыгодный для освоения.

- Самая качественная модель получилась в регионе r1. В регионах r0 и r2 RMSE большой(почти пловина среднего предсказанного).

# 3. Подготовка к расчёту прибыли

- 3.1. Все ключевые значения для расчётов сохраним в отдельных переменных.

In [ ]:
BUDGET = 10000
#BUDGET = 1e10                             # Бюджет на один регион
REVENUE_PER_PRODUCT = 0.45
#REVENUE_PER_PRODUCT = 45e4                # Доход с одной единицы продукта
N_POINTS_ALL = 500                        # Количество исследуемых точек
N_POINTS_BEST = 200                       # Количество точек для расчета прибыли
RISK_TRESHOLD = 0.025                     # Максимальный уровень убытков
BUDGET_PER_POINT = BUDGET/N_POINTS_ALL    # Бюджет на одну скважину

- 3.2. Рассчитаем достаточный объём сырья для безубыточной разработки новой скважины и сравним полученный объём сырья со средним запасом в каждом регионе.


In [ ]:
product_volume_all = BUDGET/REVENUE_PER_PRODUCT
product_per_well = product_volume_all/N_POINTS_BEST
print('Необходимое количество сырья в регионе', product_volume_all)
print('Необходимое количество сырья на скважину', product_per_well)


Необходимое количество сырья в регионе 22222.222222222223
Необходимое количество сырья на скважину 111.11111111111111


In [ ]:
print('Средний объем сырья на скважину в регионе r0', target_0.mean())
print('Средний объем сырья на скважину в регионе r1', target_1.mean())
print('Средний объем сырья на скважину в регионе r2', target_2.mean())

Средний объем сырья на скважину в регионе r0 92.49724383138823
Средний объем сырья на скважину в регионе r1 68.82446011217837
Средний объем сырья на скважину в регионе r2 94.99751009410284


## Вывод

 - Средняя концентрация продукта во всех регионах ниже необходимого значения, чтобы окупить вложения на разработку. Поэтому нельзя выбирать точки случайно.

# 4. Расчёт прибыли и рисков 

- 4.1. Напишем функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [ ]:
def revenue(target, predictions):
    pred_sorted = predictions.sort_values(ascending=False)
    selected = target[pred_sorted.index][:N_POINTS_BEST]
    profit = selected.sum()*REVENUE_PER_PRODUCT - BUDGET
    
    return profit

- 4.2. Примените технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли. Найдем среднюю прибыль, 95%-й доверительный интервал и риск убытков.

In [ ]:
def mean_revenue_and_std(target, predictions, N_POINTS_ALL=500, N_POINTS_BEST=200):    

    profit_values = []
    for i in range(1000):
        # берем выборки из данных по 500 элементов каждой выручке
        target_sample = target.sample(n=N_POINTS_ALL , replace=True, random_state = state)
        # берем соответствующие выборке предсказания нашей модели
        predict_sample = predictions[target_sample.index]
        # прогоняем по нашей функции прибыли каждую из 1000 выборок и записываем в общий массив
        profit_values.append(round(revenue(target_sample, predict_sample)))
        
    profit_values = pd.Series(profit_values)
    mean = profit_values.mean()
    confidence_interval = (profit_values.quantile(0.025), profit_values.quantile(0.975))
    loss = (profit_values < 0).mean()
    roi_coef = abs((BUDGET-mean)/mean)
    
    print('Средняя прибыль:', profit_values.mean())
    print('95% доверительный интервал:', confidence_interval)
    print('Риск убытков:', loss*100,'%')
    print('Коэффициент возврата инвестиций', round(roi_coef))
    return profit_values, mean, confidence_interval, loss, roi_coef

In [ ]:
reg = -1
revenue_df = pd.DataFrame(columns=['revenue', 'confidence_interval', 'loss', 'ROI'])

for target, pred in zips:
    reg+=1
    print('\nРегион '+str(reg)+':')
    profit_values, mean, conf, loss, roi = mean_revenue_and_std(target, pred);
    revenue_df.loc[reg] = [mean, conf, loss, roi]
        
pd.options.display.float_format = '{:,.2f}'.format
revenue_df


Регион 0:
Средняя прибыль: 456.165
95% доверительный интервал: (-78.1, 968.2999999999997)
Риск убытков: 5.2 %
Коэффициент возврата инвестиций 21

Регион 1:
Средняя прибыль: 461.375
95% доверительный интервал: (47.900000000000006, 897.1999999999998)
Риск убытков: 1.6 %
Коэффициент возврата инвестиций 21

Регион 2:
Средняя прибыль: 409.074
95% доверительный интервал: (-117.27499999999998, 998.025)
Риск убытков: 7.8 %
Коэффициент возврата инвестиций 23


,revenue,confidence_interval,loss,ROI
0,456.17,"(-78.1, 968.2999999999997)",0.05,20.92
1,461.38,"(47.900000000000006, 897.1999999999998)",0.02,20.67
2,409.07,"(-117.27499999999998, 998.025)",0.08,23.45


## Вывод

- Для разработки выбираем Регион 1. Здесь ниже всего риск убытков - меньше 2.5% и самая высокая прогнозируемая прибыль.